In [1]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.1.zip
!unzip v0.9.1.zip
!cd fastText-0.9.1
!make
!pip install .

In [2]:
from IPython import embed
import fasttext
import sys
import os
import nltk
nltk.download('punkt')
import csv
import datetime
from bs4 import BeautifulSoup
import re
import itertools

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!

In [3]:
help(fasttext.FastText)

Help on module fasttext.FastText in fasttext:

NAME
 fasttext.FastText

DESCRIPTION
 # Copyright (c) 2017-present, Facebook, Inc.
 # All rights reserved.
 #
 # This source code is licensed under the MIT license found in the
 # LICENSE file in the root directory of this source tree.

FUNCTIONS
 cbow(*kargs, **kwargs)
 
 eprint(cls, *args, **kwargs)
 
 load_model(path)
 Load a model given a filepath and return a model object.
 
 read_args(arg_list, arg_dict, arg_names, default_values)
 
 skipgram(*kargs, **kwargs)
 
 supervised(*kargs, **kwargs)
 
 tokenize(text)
 Given a string of text, tokenize it and return a list of tokens
 
 train_supervised(*kargs, **kwargs)
 Train a supervised model and return a model object.
 
 input must be a filepath. The input text does not need to be tokenized
 as per the tokenize function, but it must be preprocessed and encoded
 as UTF-8. You might want to consult standard preprocessing scripts such
 as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html
 
 The input file must must contain at least one label per line. For an
 example consult the example datasets which are part of the fastText
 repository such as the dataset pulled by classification-example.sh.
 
 train_unsupervised(*kargs, **kwargs)
 Train an unsupervised model and return a model object.
 
 input must be a filepath. The input text does not need to be tokenized
 as per the tokenize function, but it must be preprocessed and encoded
 as UTF-8. You might want to consult standard preprocessing scripts such
 as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html
 
 The input field must not contain any labels or use the specified label prefix
 unless it is ok for those words to be ignored. For an example consult the
 dataset pulled by the example script word-vector-example.sh, which is
 part of the fastText repository.

DATA
 BOW = '<'
 EOS = '</s>'
 EOW = '>'
 absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...
 division = _Feature((2, 2, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0), 8192...
 print_function = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', 0)...
 unicode_literals = _Feature((2, 6, 0, 'alpha', 2), (3, 0, 0, 'alpha', ...
 unsupervised_default = {'bucket': 2000000, 'dim': 100, 'epoch': 5, 'la...

FILE
 /databricks/python/lib/python3.6/site-packages/fasttext/FastText.py

In [4]:
# File location and type
file_location = "/FileStore/tables/training_1600000_processed_noemoticon-efba6.csv"
file_type = "csv"

# The applied options are for CSV files. For other file types, these will be ignored.
sentiment140 = spark.read.format(file_type).load(file_location).toPandas()
sentiment140.columns = ['sentiment','id','date','flag','user','text']

sentiment140.head()

,sentiment,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

Using TensorFlow backend.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data] Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [6]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

In [7]:
### SAMPLE TEST - IGNORE THIS BLOCK
sent = preprocess_text("Artificial intelligence, is the most advanced technology of the present era")
print(sent)


final_corpus = [preprocess_text(sentence) for sentence in artificial_intelligence if sentence.strip() !='']

word_punctuation_tokenizer = nltk.WordPunctTokenizer()
word_tokenized_corpus = [word_punctuation_tokenizer.tokenize(sent) for sent in final_corpus]

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-892730563453008> in <module> () 
 3 
 4 
 ----> 5 final_corpus = [ preprocess_text ( sentence ) for sentence in artificial_intelligence if sentence . strip ( ) != '' ] 
 6 
 7 word_punctuation_tokenizer = nltk . WordPunctTokenizer ( ) 

 NameError : name 'artificial_intelligence' is not defined

In [8]:
text = [preprocess_text(a) for b in sentiment140[['text']].values.tolist() for a in b]

In [9]:
text[:5]

Out[61]: ['switchfoot http twitpic 2y1zl awww bummer shoulda david carr third',
 'upset update facebook texting might result school today also blah',
 'kenichan dived many time ball managed save rest bound',
 'whole body feel itchy like fire',
 'nationwideclass behaving']

#### Converting the labels to the required format

In [11]:
sentiment140.sentiment.value_counts()

Out[47]: 4 800000
0 800000
Name: sentiment, dtype: int64

In [12]:
labels = [a for b in sentiment140[['sentiment']].values.tolist() for a in b]

In [13]:
labels[:5]

Out[53]: ['0', '0', '0', '0', '0']

In [14]:
required_labels = list()
for label in labels:
  if label=='0':
    required_labels.append('__label__negative')
  else:
    required_labels.append('__label__positive')
#   labels = ['__label__'+ s for s in labels if s=='0' then ]

In [15]:
required_labels[:10]

Out[60]: ['__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative',
 '__label__negative']

In [16]:
import pandas as pd
required_data = pd.DataFrame(columns=['reviews_score', 'text'])
required_data['text'] = text
required_data['reviews_score'] = required_labels
required_data.head()

,reviews_score,text
0,__label__negative,switchfoot http twitpic 2y1zl awww bummer shou...
1,__label__negative,upset update facebook texting might result sch...
2,__label__negative,kenichan dived many time ball managed save res...
3,__label__negative,whole body feel itchy like fire
4,__label__negative,nationwideclass behaving


In [17]:
len(required_data)

Out[67]: 1600000

In [18]:
import numpy as np
msk = np.random.rand(len(required_data)) < 0.8

train = required_data[msk]

test = required_data[~msk]

print(len(train), len(test))

1280127 319873

In [19]:
train.to_csv(r'/dbfs/FileStore/stanford_16k_fasttextTrain.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

In [20]:
test.to_csv(r'/dbfs/FileStore/stanford_16k_fasttextTest.txt', index=False, sep=' ', header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

##### Building fastText Module

In [22]:
import fasttext
model = fasttext.train_supervised(input="/dbfs/FileStore/stanford_16k_fasttextTrain.txt", epoch=25, lr=0.4 ,wordNgrams=2, loss='hs')

In [23]:
model.test("/dbfs/FileStore/stanford_16k_fasttextTest.txt")

Out[19]: (319873, 0.7333066560791314, 0.7333066560791314)

In [24]:
model.predict('The drone can fly very high but very costly', k=2)

Out[15]: (('__label__positive', '__label__negative'), array([0.56306833, 0.4369517 ]))

In [25]:
model.predict('The drone is good but very costly')

Out[12]: (('__label__negative',), array([0.89230967]))

In [26]:
model.predict('The drone is very good but cheap')

Out[13]: (('__label__positive',), array([1.0000037]))

In [27]:
# print(model.words)
print(model.labels)

['__label__positive', '__label__negative']

In [28]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('/dbfs/FileStore/stanford_16k_fasttextTest.txt'))

N	319873
P@1	0.733
R@1	0.733

In [29]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from tldextract import extract
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
spark = SparkSession \
        .builder \
        .appName("Twitter Data Loading") \
        .config("spark.gars.config.option", "gars-config") \
        .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [30]:
df= spark.table('topictags_csv').toPandas()

/databricks/python/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [31]:
df.columns

Out[12]: Index(['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7', '_c8', '_c9',
 '_c10', '_c11', '_c12', '_c13'],
 dtype='object')

,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13
0,None,id_str,text,text_ht,textht_td,ner_text,domain,external_url,textTopic,scoreTextTopic,textHT_Topic,scoreHT_Topic,descTopic,scoreDescTopic
1,0,1185344422378180609,gadget waterproof outdoor solar night light te...,gadget waterproof outdoor solar night light te...,gadget waterproof outdoor solar night light te...,GADGET Waterproof Outdoor Solar Night Light te...,[],['https://www.gadgetcetera.com/product/gadget-...,1,0.79338074,1,0.83518404,1,0.551684
2,1,1185342993554907138,super offer drones and accessories visit take ...,super offer drones and accessories visit take ...,super offer drones and accessories visit take ...,Super Offer Drones and accessories visit Take ...,[],['https://dronesaccessories.com/s/DroneXPRO'],1,0.8077339,1,0.79964167,1,0.8164825
3,2,1185341343993864194,car charger for dji mavic pro dji drones drone...,car charger for dji mavic pro dji drones drone...,car charger for dji mavic pro dji drones drone...,Car Charger for DJI Mavic Pro dji drones drone...,[],None,1,0.738107,1,0.7370199,1,0.7182667
4,3,1185341053852934144,as organizations innovate and deploy new techn...,as organizations innovate and deploy new techn...,as organizations innovate and deploy new techn...,As organizations innovate and deploy new techn...,[],['http://ow.ly/JBm330pK1Ob'],2,0.8235579,3,0.46484897,2,0.56759185


In [33]:
topic_df=df[['_c1','_c3','_c12' ]]
topic_df.head()

,_c1,_c3,_c12
0,id_str,text_ht,descTopic
1,1185344422378180609,gadget waterproof outdoor solar night light te...,1
2,1185342993554907138,super offer drones and accessories visit take ...,1
3,1185341343993864194,car charger for dji mavic pro dji drones drone...,1
4,1185341053852934144,as organizations innovate and deploy new techn...,2


In [34]:
new_header = topic_df.iloc[0] #grab the first row for the header
topic_df.columns = new_header #set the header row as the df header
topic_df = topic_df[1:] #take the data less the header row

In [35]:
topic_df.head()

,id_str,text_ht,descTopic
1,1185344422378180609,gadget waterproof outdoor solar night light te...,1
2,1185342993554907138,super offer drones and accessories visit take ...,1
3,1185341343993864194,car charger for dji mavic pro dji drones drone...,1
4,1185341053852934144,as organizations innovate and deploy new techn...,2
5,1185339289212801024,is updating year old mapping technology and de...,3


In [36]:
for i in range(1, len(mylist)):
  df2.loc[i,["sentiment_score"]]=model.predict(i, k=2)
  

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2769566558784178> in <module> () 
 1 for i in range ( len ( mylist ) ) : 
 ----> 2 df2 . loc [ i , [ "sentiment_score" ] ] = model . predict ( i , k = 2 ) 
 3 

 /databricks/python/lib/python3.6/site-packages/fasttext/FastText.py in predict (self, text, k, threshold, on_unicode_error) 
 153 return result_as_pair [ 'label' ] . tolist ( ) , result_as_pair [ 'probability' ] 
 154 else : 
 --> 155 text = check ( text ) 
 156 predictions = self . f . predict ( text , k , threshold , on_unicode_error ) 
 157 probs , labels = zip ( * predictions ) 

 /databricks/python/lib/python3.6/site-packages/fasttext/FastText.py in check (entry) 
 138 
 139 def check ( entry ) : 
 --> 140 if entry . find ( '\n' ) != - 1 : 
 141 raise ValueError(
 142 "predict processes one line at a time (remove \'\\n\')" 

 AttributeError : 'int' object has no attribute 'find'

In [37]:
topic_df.loc[:,['text_ht']]

,text_ht
1,gadget waterproof outdoor solar night light te...
2,super offer drones and accessories visit take ...
3,car charger for dji mavic pro dji drones drone...
4,as organizations innovate and deploy new techn...
5,is updating year old mapping technology and de...
6,reminder please keep your drones at home remem...
7,check out our founder jae brown taking his tim...
8,ancient greek terra cotta bowl corinthian pott...
9,top holy stone mini drones and predator drone ...
10,neat story on and drone racing from friends at...


In [38]:
## topic 1

In [39]:
sample_df_temp = topic_df[topic_df.descTopic=='1'].reset_index()
sample_df1 = sample_df_temp.loc[0:49,:]
sample_df1.shape

Out[151]: (50, 4)

In [40]:
temp=list()
for item in range(50):
  temp.append(model.predict(list(sample_df1.loc[item,['text_ht']])[0],k=2)[1][0])

In [41]:
temp

Out[153]: [0.9002419710159302,
 0.6659582257270813,
 0.9765505790710449,
 0.9907771944999695,
 0.9638704061508179,
 0.6265354752540588,
 0.9139151573181152,
 0.7857284545898438,
 0.9997010231018066,
 0.9139151573181152,
 0.872802734375,
 0.9893723130226135,
 0.8648180961608887,
 0.6659582257270813,
 0.8066430687904358,
 0.8433306217193604,
 0.6004169583320618,
 0.6529549360275269,
 0.9139151573181152,
 0.9822624921798706,
 0.9991928935050964,
 0.6652733087539673,
 0.9942998886108398,
 0.9716125130653381,
 0.9139151573181152,
 0.7601130604743958,
 0.9991775155067444,
 0.9792426228523254,
 0.5190051198005676,
 0.9139151573181152,
 0.9839230179786682,
 0.998567521572113,
 0.9998601675033569,
 0.966505229473114,
 0.9902994632720947,
 0.9804791808128357,
 0.999862551689148,
 0.9373852610588074,
 0.995548665523529,
 0.6659582257270813,
 0.9924934506416321,
 0.988955557346344,
 0.7935492396354675,
 0.763626754283905,
 0.8322154879570007,
 0.9139151573181152,
 0.9997313618659973,
 0.5309649705886841,
 0.5147765278816223,
 0.728874921798706]

In [42]:
sample_df1["sentiment_score"]=temp

/local_disk0/tmp/1584038087852-0/PythonShell.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import absolute_import

In [43]:
sample_df2.head()

,index,id_str,text_ht,descTopic
0,4,1185341053852934144,as organizations innovate and deploy new techn...,2
1,23,1185329170424840192,drone delivery is one step closer to reality d...,2
2,27,1185327691639095296,drone delivery is one step closer to reality d...,2
3,42,1185315878084206592,walmart seeks to deliver packages without touc...,2
4,44,1185315154818457602,breakthrough day for drones and,2


In [44]:
sample_df1.drop(['index'], axis=1).to_csv("/dbfs/FileStore/sample_df1.csv")

In [45]:
# topic 2

In [46]:
sample_df_temp2 = topic_df[topic_df.descTopic=='2'].reset_index()
sample_df2 = sample_df_temp2.loc[0:49,:]
sample_df2.shape

Out[155]: (50, 4)

In [47]:
temp=list()
for item in range(50):
  temp.append(model.predict(list(sample_df2.loc[item,['text_ht']])[0],k=2)[1][0])
sample_df2["sentiment_score"]=temp

/local_disk0/tmp/1584038087852-0/PythonShell.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import errno

In [48]:
sample_df2.drop(['index'], axis=1).to_csv("/dbfs/FileStore/sample_df2.csv")

In [49]:
# topic 3

In [50]:
sample_df_temp3 = topic_df[topic_df.descTopic=='3'].reset_index()
sample_df3 = sample_df_temp3.loc[0:49,:]
sample_df3.shape

Out[157]: (50, 4)

In [51]:
temp=list()
for item in range(50):
  temp.append(model.predict(list(sample_df3.loc[item,['text_ht']])[0],k=2)[1][0])
sample_df3["sentiment_score"]=temp
sample_df3.head()

,index,id_str,text_ht,descTopic,sentiment_score
0,5,1185339289212801024,is updating year old mapping technology and de...,3,0.669379
1,10,1185334753806807040,neat story on and drone racing from friends at...,3,0.998929
2,12,1185334023301816320,one of the best parts of mc week was the drone...,3,0.946886
3,18,1185331017944588289,drones can cover approximately acres in one fl...,3,0.861552
4,20,1185330697013059584,somebody call peter parker drones safety damn,3,0.999833


In [52]:
sample_df3.drop(['index'], axis=1).to_csv("/dbfs/FileStore/sample_df3.csv")

In [53]:
# topic 4

In [54]:
sample_df_temp4 = topic_df[topic_df.descTopic=='4'].reset_index()
sample_df4= sample_df_temp4.loc[0:49,:]
sample_df4.shape

Out[159]: (50, 4)

In [55]:
temp=list()
for item in range(50):
  temp.append(model.predict(list(sample_df4.loc[item,['text_ht']])[0],k=2)[1][0])
sample_df4["sentiment_score"]=temp
sample_df4.head()

,index,id_str,text_ht,descTopic,sentiment_score
0,6,1185339157390004224,reminder please keep your drones at home remem...,4,0.823058
1,13,1185333669000466432,heavy lifting drones from pakistan were used t...,4,0.685783
2,26,1185328024360804354,member of the kurdish red crescent convoy of t...,4,0.506751
3,37,1185319822059401216,has deployed drones detection system providing...,4,0.952686
4,38,1185318643762110464,drones being used in forestry expand their use...,4,0.993455


In [56]:
sample_df4.drop(['index'], axis=1).to_csv("/dbfs/FileStore/sample_df4.csv")

In [57]:
# topic 5

In [58]:
sample_df_temp5 = topic_df[topic_df.descTopic=='5'].reset_index()
sample_df5= sample_df_temp5.loc[0:49,:]
sample_df5.shape

Out[161]: (50, 4)

In [59]:
temp=list()
for item in range(50):
  temp.append(model.predict(list(sample_df5.loc[item,['text_ht']])[0],k=2)[1][0])
sample_df5["sentiment_score"]=temp
sample_df5.head()

,index,id_str,text_ht,descTopic,sentiment_score
0,7,1185338478688690176,check out our founder jae brown taking his tim...,5,0.972570
1,11,1185334296544010240,deadman hill overlook mi mavic pro drone uav,5,0.670654
2,14,1185333447327281157,we hope the participants discussed future read...,5,0.997522
3,15,1185333260101963776,we hope the participants discussed future read...,5,0.997522
4,16,1185333028203184128,flew bit over brainerd minnesota shot it from ...,5,0.935394


In [60]:
sample_df5.drop(['index'], axis=1).to_csv("/dbfs/FileStore/sample_df5.csv")

In [61]:
sample_df=pd.concat([sample_df1,sample_df2,sample_df3,sample_df4,sample_df5]).drop(['index'], axis=1)


In [62]:
sample_df.to_csv("/dbfs/FileStore/tables/sample_df.csv")

In [63]:
display(sample_df)

id_str,text_ht,descTopic,sentiment_score
1185344422378180609,gadget waterproof outdoor solar night light technology tech gifts drones earbuds nerd gadgets android phone wireless bluetooth,1,0.9002419710159302
1185342993554907138,super offer drones and accessories visit take breathtaking videos with this drone pro best seller drone pro,1,0.6659582257270813
1185341343993864194,car charger for dji mavic pro dji drones drone pilot drone flier drone fly,1,0.9765505790710449
1185336661749702656,ancient greek terra cotta bowl corinthian pottery vessel corinth skyphos as is drones uav,1,0.9907771944999695
1185335954900471809,top holy stone mini drones and predator drone reviews rc drones mini drones helicopter drones and quadcopters are the toast of the moment but the process of getting,1,0.9638704061508179
1185330801119879168,agree which is why am so grateful to julian assange for having the courage to expose your cheating hillary is warmonger prostate cancer on america,1,0.6265354752540588
1185327406946623489,drones and accessories visit our online store where you will find all types of professional drones drone camara,1,0.9139151573181152
1185321292188270592,gadget portable wireless mini bluetooth speaker technology tech gifts drones earbuds nerd gadgets android phone wireless bluetooth,1,0.7857284545898438
1185321159375642624,new video tutorial out check link in bio fpv drone fpv freestyle fpv life fpv racing fpv addiction fpv drone drones tag ram drone racing quad copter go pro drones team black sheep fpv addict,1,0.9997010231018066
1185311763576868864,drones and accessories visit our online store where you will find all types of professional drones drone camara,1,0.9139151573181152


#Start Here

In [66]:
import pandas as pd

In [67]:
dfTopicTags1 = pd.read_csv('/dbfs/FileStore/tables/topicTags.csv')

In [68]:
dfTopicTags = dfTopicTags1[['id_str', 'descTopic', 'text_ht']]

In [69]:
df1 = pd.read_csv('/dbfs/FileStore/consolidated.csv')
df2 = pd.read_csv('/dbfs/FileStore/consolidated2.csv')

/databricks/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,9,18,30,31,35) have mixed types. Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler, result=result)
/databricks/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,2,3,4,5,6,7,9,10,11,13,18,19,20,21,24,27,30,31,32,33,34,35,36,38) have mixed types. Specify dtype option on import or set low_memory=False.
 interactivity=interactivity, compiler=compiler, result=result)

In [70]:
df_raw1 = pd.concat([df1,df2])

/local_disk0/tmp/1584038087852-0/PythonShell.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

 from __future__ import absolute_import

In [71]:
dfRaw = df_raw1[['id_str', 'text']]

In [72]:
df = pd.merge(dfRaw, dfTopicTags, on='id_str')

In [73]:
df.shape

Out[45]: (243452, 4)

In [74]:
df.head()

,id_str,text,descTopic,text_ht
0,1.217957e+18,It's a bird! It's a plane! No it's @UPS and @C...,5,it bird it plane no it and delivering prescrip...
1,1.217956e+18,#Drones and accessories visit our online store...,1,drones and accessories visit our online store ...
2,1.217954e+18,Time flies when you’re flying.\n\n📸\n@asdetroi...,1,time flies when you re flying excel drones
3,1.217954e+18,RT @foxfuryLED: Introducing the D3060 - a smal...,3,introducing the small format drone and utility...
4,1.217950e+18,"RT @JPCarter47: BIOPRINTS\n""...Cutting edge te...",1,bioprints cutting edge technology meets classi...


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243452 entries, 0 to 243451
Data columns (total 4 columns):
id_str 243452 non-null float64
text 243452 non-null object
descTopic 243452 non-null int64
text_ht 243452 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 9.3+ MB

In [76]:
def topicTweets(numberOfTweetsPerTopic, topicTagList, requiredColumns, dataFrame, topicIndicatorColumn):

  dummyDf = pd.DataFrame()
  tempDF = pd.DataFrame()
  for topicNumber in topicTagList:
    dummyDf = dataFrame[dataFrame[topicIndicatorColumn]==topicNumber][requiredColumns]
    dummyDf = dummyDf.head(numberOfTweetsPerTopic)
    tempDF = tempDF.append(dummyDf)
  return tempDF

In [77]:
numberOfTweetsPerTopic = 25
topicTagList = [1, 2, 3, 4, 5]
requiredColumns = ['id_str', 'text', 'text_ht','descTopic']
dataFrame = df
topicIndicatorColumn = 'descTopic'

dataFrame = topicTweets(numberOfTweetsPerTopic, topicTagList, requiredColumns, dataFrame, topicIndicatorColumn)

In [78]:
dataFrame.head()

,id_str,text,text_ht,descTopic
1,1.217956e+18,#Drones and accessories visit our online store...,drones and accessories visit our online store ...,1
2,1.217954e+18,Time flies when you’re flying.\n\n📸\n@asdetroi...,time flies when you re flying excel drones,1
4,1.217950e+18,"RT @JPCarter47: BIOPRINTS\n""...Cutting edge te...",bioprints cutting edge technology meets classi...,1
10,1.217937e+18,RT @TopDroneReviews: DJI Mavic Pro 4K Flying C...,dji mavic pro flying camera review drones,1
13,1.217935e+18,RT @JPCarter47: THE O’ROURKE SERIES\nPICKUP TH...,the rourke series pickup the series today link...,1


In [79]:
model = fasttext.train_supervised(input="/dbfs/FileStore/stanford_16k_fasttextTrain.txt", epoch=25, lr=0.4 ,wordNgrams=2, loss='hs')

In [80]:
model.predict('The drone can fly very high but very costly', k=2)

Out[144]: (('__label__negative', '__label__positive'), array([0.98801303, 0.01200701]))

In [81]:
dataFrame.shape[0]

Out[145]: 125

In [82]:
tempTag = []
tempScore = []

for tweet in dataFrame['text_ht']:
  tempTag.append(model.predict(tweet,k=2)[0][0])
  tempScore.append(model.predict(tweet,k=2)[1][0])
  

In [83]:
dataFrame["sentiment score"]=tempScore
dataFrame["sentiment tag"]=tempTag

In [84]:
dataFrame = dataFrame.drop(columns='text_ht')

In [85]:
dataFrame

,id_str,text,text_ht,descTopic,sentiment score,sentiment tag
1,1.217956e+18,#Drones and accessories visit our online store...,drones and accessories visit our online store ...,1,0.897805,__label__positive
2,1.217954e+18,Time flies when you’re flying.\n\n📸\n@asdetroi...,time flies when you re flying excel drones,1,0.684260,__label__positive
4,1.217950e+18,"RT @JPCarter47: BIOPRINTS\n""...Cutting edge te...",bioprints cutting edge technology meets classi...,1,0.993815,__label__positive
10,1.217937e+18,RT @TopDroneReviews: DJI Mavic Pro 4K Flying C...,dji mavic pro flying camera review drones,1,0.678470,__label__negative
13,1.217935e+18,RT @JPCarter47: THE O’ROURKE SERIES\nPICKUP TH...,the rourke series pickup the series today link...,1,0.553313,__label__positive
20,1.217591e+18,OUR PLANET PART 3 | Powerful | Aerial Drone Vi...,our planet part powerful aerial drone video fi...,1,0.999882,__label__positive
53,1.216868e+18,Super Offer #Drones and accessories visit http...,super offer drones and accessories visit take ...,1,0.875712,__label__positive
56,1.216865e+18,RT @dronetool: (Ancient Gandharan Stucco Seate...,ancient gandharan stucco seated buddha traces ...,1,0.977089,__label__positive
58,1.216858e+18,GADGET 25- Mini Drone https://t.co/67HAbzjo04 ...,gadget mini drone technology tech gifts drones...,1,0.574102,__label__positive
62,1.216852e+18,Super Offer #Drones and accessories visit http...,super offer drones and accessories visit take ...,1,0.875712,__label__positive


In [86]:
display(dataFrame)

id_str,text,descTopic,sentiment score,sentiment tag
1.21795625951052595E18,#Drones and accessories visit our online store https://t.co/hyp5tOC3vM where you will find all types of professiona… https://t.co/4vPXvQZNcV,1,0.5413447022438049,__label__positive
1.2179540220633129E18,Time flies when you’re flying. 📸 @asdetroitsown #excel #drones https://t.co/GQpCebkwqw,1,0.5945179462432861,__label__negative
1.21794954088571699E18,"RT @JPCarter47: BIOPRINTS ""...Cutting edge technology meets classic detective fiction..."" https://t.co/iHcwyLVEPh #IARTG #Kindle #Amazon #…",1,0.9341397881507874,__label__positive
1.21793691748603494E18,RT @TopDroneReviews: DJI Mavic Pro 4K Flying Camera Review https://t.co/ENo1xWV3Tz #Drones https://t.co/b7x0GS72kt,1,0.8209832310676575,__label__negative
1.21793461977085952E18,RT @JPCarter47: THE O’ROURKE SERIES PICKUP THE SERIES TODAY LINK: https://t.co/iHcwyLVEPh #Books #IARTG #Kindle #Amazon #ReadIndie #indiea…,1,0.7118966579437256,__label__negative
1.21759122778636698E18,OUR PLANET PART 3 | Powerful | Aerial Drone Video Filmed in 6K https://t.co/AQxmn0tayM via @YouTube #Alps #Alpine… https://t.co/KW47LBe3IS,1,0.9999945163726807,__label__positive
1.21686764848046899E18,Super Offer #Drones and accessories visit https://t.co/X6qL5EEORY << == Take Breathtaking Videos with this #Drone X… https://t.co/Ij3rdSUMGJ,1,0.5696555972099304,__label__negative
1.21686476477894246E18,RT @dronetool: (Ancient Gandharan Stucco Seated Buddha w/ traces of Blue painting) - https://t.co/rndhhFSBpE #drones #UAV https://t.co/mdEO…,1,0.801601767539978,__label__positive
1.21685787002680115E18,GADGET 25- Mini Drone https://t.co/67HAbzjo04 32.95 #technology #techgifts #drones #earbuds #nerd #gadgets #android… https://t.co/MlCn3POFKe,1,0.9388210773468018,__label__negative
1.21685209192850227E18,Super Offer #Drones and accessories visit https://t.co/X6qL5EEORY << == Take Breathtaking Videos with this #Drone X… https://t.co/2CsmoMsn2i,1,0.5696555972099304,__label__negative
